In [138]:
#IMPORTS

import enchant  #pip install pyenchant
import sys,os
from os import path
import getpass
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import itertools
import ast
import nltk #pip install nltk
            #nltk.download('stopwords')
from nltk.corpus import stopwords
import geonamescache  #pip install geonamescache
import geopandas as gpd #pip install geopandas or conda install -c conda-forge geopandas AND pip install descartes
from geopy.geocoders import Nominatim #pip install geopy and pip install Nominatim
import itertools
import math

# PRE FUNCIONES

In [4]:
def addToDic(d,key,value, how = None):
    
    """ 
    RECIBE: Un diccionario, una key y un value sumable con otros values y una forma de suamarlos
    DEVUELVE: Nada porque el diccionario cambia globalmente al cambiarse aca
    """
    
    if how == None:
        if key in d:
            d[key] += value
        else:
            d[key] = value
    else:
        if key in d:
            d[key] = how(d[key],value)
        else:
            d[key] = value

In [140]:
def mergeListOfLists(l):

    """ 
    RECIBE: Una lista de listas
    DEVUELVE: Una lista que contiene como elementos los elementos de cada lista de la lista de listas
    """

    return list(itertools.chain.from_iterable(l))

In [141]:
def mapNameLengToGeographical(geographicals):
    
    """ 
    RECIBE: Una iterable de cosas geograficas del geonamescache
    DEVUELVE: Un diccionario que tiene como k longitud del nombre 
                y como una lista de nombres con esa longitud
    """
    
    dic = {}
    
    for n in range(MAX_LOCATION_LENGTH_IN_WORDS):
        for geographical in geographicals:

            geographicalName = geographicals[geographical]['name']
            nameParts = geographicalName.split(' ')
            if len(nameParts) == n:
                addToDic(dic,n,[geographicalName])
    return dic

In [142]:
def getAcronim(text):
    
    """ 
    RECIBE: Un texto
    DEVUELVE: Devuelve un string solo con las mayusculas
    """
    
    words = text.split(' ')
    
    acronim = ''
    for word in words:
        if word[0].isupper():
            acronim += word[0]
    return acronim 

In [6]:
def nonDuplicateListSum(a,b):
    
    """ 
    RECIBE: Dos listas
    DEVUELVE: La suma sin repeticion de las listas donde los 
                unicos elementos repetidos posibles de la lista resultado 
                son lo que ya tenia a desde antes
    """
    
    for e in b:
        if not e in a:
            a.append(e)
    return a

# VARIABLES GLOBALES

In [143]:
#GLOBALES

englishDictionary = enchant.Dict("en_US")
stopWords = set(stopwords.words('english'))
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','ñ','o','p','q','r','s','t','u',
           'v','x','y','z']

#WORD TYPE FLAGS

EMPTY = '__EMPTY'
WEB_ADDRESS = '__WEB_ADDRESS'
ACRONIM = '__ACRONIM'
NUMERIC = '__NUMERIC'
NON_ALFABETICAL = '__NON_ALFABETICAL'


# GEOGRAPHIC

gc = geonamescache.GeonamesCache()

MAX_LOCATION_LENGTH_IN_WORDS = 8 # no hay ciudades, paises o continentes cuyo nombre supere las 7 palabras

continents = gc.get_continents()
countries = gc.get_countries()
usStates = gc.get_us_states()
cities = gc.get_cities()

citiesAndUsStatesNames = [cities[code]['name'] for code in cities] + [usStates[code]['name'] for code in usStates]
countriesNames = [countries[code]['name'] for code in countries] + ['England','Ireland','Scotland', 'Brasil']
continentsNames = [continents[code]['name'] for code in continents]

citiyOrUSStateNameToCountryMap = {}

for key in usStates:
    stateName = usStates[key]['name']
    addToDic(citiyOrUSStateNameToCountryMap,stateName,['United States'])
    
for key in cities:
    cityName = cities[key]['name']
    addToDic(citiyOrUSStateNameToCountryMap,cityName,[countries[cities[key]['countrycode']]['name']])

countryNameContinentNameMap = {}

for k in countries:
    
    addToDic(countryNameContinentNameMap,countries[k]['name'],continents[countries[k]['continentcode']]['name'])

addToDic(countryNameContinentNameMap,'England','Europe')
addToDic(countryNameContinentNameMap,'Ireland','Europe')
addToDic(countryNameContinentNameMap,'Scotland','Europe')
addToDic(countryNameContinentNameMap,'Brasil','South America')
    
cityOrUSStateAcronimToNameMap = {}

for acr in usStates:
    addToDic(cityOrUSStateAcronimToNameMap,acr,[usStates[acr]['name']])

for k in cities:
    acr = getAcronim(cities[k]['name'])
    addToDic(cityOrUSStateAcronimToNameMap,acr,[cities[k]['name']])
    
addToDic(cityOrUSStateAcronimToNameMap,'L.A',['United States'])
    
    
cityOrStateToCountryMap = {}

for acr in usStates:
    addToDic(cityOrStateToCountryMap,usStates[acr]['name'],['United States'])

for k in cities:
    addToDic(cityOrStateToCountryMap,cities[k]['name'],[countries[cities[k]['countrycode']]['name']])
    
countryAcronimToCountryName = {}

for acr in countries:
    addToDic(countryAcronimToCountryName,acr,countries[acr]['name'])
    addToDic(countryAcronimToCountryName,countries[acr]['iso3'],countries[acr]['name'])
    
addToDic(countryAcronimToCountryName,'U.S','United States')
        
UNKNOWN = '__UNKNOWN'
CITY = '__CITY'
COUNTRY = '__COUNTRY'
CONTINENT = '__CONTINENT'

# PROCESAMIENTO DE TEXTO

In [144]:
def extractWordAfterSymbol(text,symbol):
    
    """ 
    Generalizando, como hizo mauro
    
    RECIBE: Un texto plano que puede tener cualquier cosa
    DEVUELVE: Una lista de todos los strings que estaban precedidos por un simbolo(sea @, #, etc)
    EJEMPLO: 'Hola que tal soy @axel, y vivo en @capitalFederal' -----> [axel,capitalFederal]'
    """
    
    words = text.split(' ')
    
    users = []
    userToSave = ""
    for word in words:
        if len(word) != 0 and symbol in word:
            splittedWord = word.split(symbol)
            if len(splittedWord) > 1:
                userToSave = splittedWord[1]
            else:
                userToSave = splittedWord[0]
        if userToSave != '':
            users.append(userToSave)
            userToSave = ""
    return users

In [145]:
def hasCertainWords(stuffWithWordsInIt,words):
    
    """ 
    RECIBE: Recibe algo que tiene palabras dentro (texto, lista de palabras, conjuntos con palabras, etc)
    DEVUELVE: Devuelve True o False dependiendo de si el el contendor contiene ALGUNA palabra en la lista de palabras
    """ 
    
    for word in words:
        if word in stuffWithWordsInIt:
            return True
    return False

In [146]:
def acortar_texto(df):
    """
    RECIBE: un df
    DEVUELVE: el mismo df con una columna con version corta de la columna 'text'
    """
    
    
    df['short_text'] = 'None'
    cant_filas = len(df.index)
    for y in range(cant_filas):
        df['short_text'][y:]= df.iloc[y][0][:15] + "..."

In [147]:
def hasQuestion(text):
    '''
    RECIBE: Un string.
    DEVUELVE: True si tiene al menos una palabra en ingles seguida por al menos un signo de pregunta, sino retorna FALSE
    '''
    text = text.replace('?', ' ? ')
    englishDictionary = enchant.Dict("en_US")
    text = takeOutSpecialCaractersFromText(text, specialCharacters = [',','.','#','!','%','*','(',')','"','-','_', '$', ':', '/', '\''])
    textSplitted = text.split()
    for word in textSplitted:
        if englishDictionary.check(word):
            for j in range(textSplitted.index(word) + 1, len(textSplitted)):
                if ('?' in textSplitted[j]):
                    return True
            return False
    return False

In [148]:
def takeOutSpecialCaractersFromText(text, specialCharacters = None):
    '''
    RECIBE: Un string.
    DEVUELVE: El string sin caracteres especiales. Con el parametro specialCharacters se puede decidir que caracteres sacar.
    '''
    if specialCharacters == None:
        specialCharacters = ['&','\n','<','>','=',',','.','#','!','%','*','(',')','"','-','_', '?', '$', ':', '/','\''] + [str(i) for i in range(0,10)]
    
    
    for char in specialCharacters:
        text = text.replace(char, ' ')
    return text

In [149]:
def separateOutBiaSpecialCaractersAndMayuscFromText(text, specialCharacters = None, notRemove = []):
    '''
    RECIBE: Un string.
    DEVUELVE: Una lista de los substrings que fuerons separados por los caracteres especiales o las mayusuclas.
               Con el parametro specialCharacters se puede decidir que caracteres sacar.
    '''
    if specialCharacters == None:
        specialCharacters = ['&','\n','<','>','=',',','.','#','!','%','*','(',')',
                             '"',' ','-','_', '?', '$', ':', '/','\''] + [str(i) for i in range(0,10)]
        specialCharacters[specialCharacters.index("'")] = '&' #HARDCODEADISIMO
    
    words = []
    subWord = ''
    prevC = ''
    
    for c in text:      
        if c in specialCharacters and not c in notRemove:
            if subWord != '':
                words.append(subWord)
                subWord = ''
        else:
            subWord += c
        
    if subWord != '':
        words.append(subWord)
        
    return words

In [150]:
def extractTextFrom(data):

    """ 
    RECIBE: El data frame de tweets
    TENER EN CUENTA: 
            Considera el campo location, keyword y text
            No altera al data frame
    DEVUELVE: En una lista como elementos la concatenacion de campo keyword, location y text
    """
    data_copy = data.copy()
    data_copy['keyword'].fillna('', inplace = True)
    data_copy['location'].fillna('', inplace = True)
    
    return list(data_copy['keyword'] + ' ' + data_copy['location'] + ' ' + data_copy['text'])

In [151]:
def extractTextFormTweet(tweet):

    """ 
    RECIBE: Un tweet
    TENER EN CUENTA: 
            Considera el campo location, keyword y text
            No altera al data frame
    DEVUELVE: Un texto que contiene la keyword, la location y el texto del tweet
    """    
    return tweet[3] + ' ' + tweet[1] + ' ' + tweet[2]

In [152]:
def isWebAddres(token):
    
    """ 
    RECIBE: Un token
    DEVUELVE: True o False dependiendo de si contiene el string http
    """  
    
    return 'http' in token

In [153]:
def isAcronim(token):
    
    """ 
    RECIBE: Un token
    DEVUELVE: True o False dependiendo de si el token tiene pinta de acronimo
    """  
    
    MAX_ACRONIM_LEN = 5
    MIN_ACRONIM_LEN = 2
    
    if len(token) <= MAX_ACRONIM_LEN and len(token) >= MIN_ACRONIM_LEN:
        
        allUpper = True
        for c in token:
            if not c.isupper():
                allUpper = False
        
        return allUpper
    
    else:
        return False

In [154]:
def allNumeric(token):
    """ 
    RECIBE: Un token
    DEVUELVE: True o False dependiendo de si todos los caracteres son numericos
    EJEMPLO: 'hol4' -----> False
    """    
    allNumeric = True
    for c in token:
        if not c.isnumeric():
            allNumeric = False
    return allNumeric

In [155]:
def hasNumeric(token):
    """ 
    RECIBE: Un token cualquiera
    DEVUELVE: True o False dependiendo de si contiene algun numero
    EJEMPLO: 'hol4' -----> True
    """    
    
    for c in token:
        if c.isnumeric():
            return True
    return False

In [156]:
def nonAlfabetical(token):

    """ 
    RECIBE: Un token
    DEVUELVE: True o False dependiendo de si todo los caracteres son no alfabeticos
    """  
    
    allNotAlphabetical = True
    for c in token:
        if c in alphabet:
            allNotAlphabetical = False
        
    return allNotAlphabetical

In [157]:
def tokenize(text):
    
    """ 
    RECIBE: Texto crudo
    DEVUELVE: Una lista de 'palabras' (tokens) que por su 'significado' se supone que son entidades
              separadas
    """
    
    return separateOutBiaSpecialCaractersAndMayuscFromText(str(text))

In [158]:
def cleanUpToken(token):
    
    """ 
    RECIBE: Un token
    DEVUELVE: Una lista de palabras que surgen de limpiar el token o un flag que especifica que tipo de token es
    """  
    
    if token == '':
        return [EMPTY]
    
    if isWebAddres(token):
        return [WEB_ADDRESS]

    if isAcronim(token):
        return [ACRONIM]
    
    if allNumeric(token):
        return [NUMERIC]
    
    if nonAlfabetical(token):
        return [NON_ALFABETICAL]
    
    subTokens = [t.lower() for t in separateOutBiaSpecialCaractersAndMayuscFromText(token)]        

    if len(subTokens) == 1:
        return subTokens
    
    for subToken in subTokens:
        return cleanUpToken(subToken)
    

In [11]:
def cleanUpTokens(textsTokens):
    
    """ 
    RECIBE: Una lista de listas de tokens
    DEVUELVE: Limpia token a token para transformarlos en palabras validas segun cleanUpToken
             y devuelve una lista (conservando repeticiones) de palabras validas
             un map de palabras valida ----> [token1,token2,...,tokenk]
             un map de token---->palabra valida
    """  

    cleanWords = []
    tokenCleanMap = {}
    cleanTokenMap = {}
    
    for tokens in textsTokens:
        
        for token in tokens:
            
            words = cleanUpToken(token)
            
            cleanWords += words
            addToDic(tokenCleanMap,token,words)
            
            for word in words:
                
                addToDic(cleanTokenMap,word,[token])
            #print("{} -----> {}".format(token,words))
    
    return cleanWords, tokenCleanMap, cleanTokenMap

# FUNCIONES DE INGLES

In [160]:
def getEnlgishWord(word):
       
    """
    RECIBE: Un texto plano que puede tener cualquier cosa
    TENER EN CUENTA: Si la palabra es vacia retorna la palabra vacia
                     No distingue mayusculas
    DEVUELVE: La palabra en ingles mas parecida
    """  
    failure = False
    
    if word == None:
        failure = True
        
    if word == '':
        failure = True
        
    else:
        englishWord = getMaxSimilarityWord(word,englishDictionary.suggest(word))[0]
    
    if failure:  
        return word
    else:
        return englishWord
        

In [161]:
def hasEnglishWord(text):
    """
    RECIBE: Un string.
    DEVUELVE: True si tiene al menos una palabra en ingles, sino retorna FALSE
    """
    englishDictionary = enchant.Dict("en_US")
    text = takeOutSpecialCaractersFromText(text)
    textSplitted = text.split()
    for word in textSplitted:
        if englishDictionary.check(word):
            return True
    return False

In [162]:
def englishPercentage(text):
    text = text.lower()
    text = takeOutSpecialCaractersFromText(text)
    englishDictionary = enchant.Dict("en_US")
    textSplitted = text.split()
    cantidadDePalabrasEnIngles = 0
    for word in textSplitted:
        if englishDictionary.check(word):
            cantidadDePalabrasEnIngles += 1
    return (cantidadDePalabrasEnIngles / len(textSplitted)) * 100

# VARIOS

In [163]:
def mapBinaryLabel(binaryIndicator, labelsList):
    
    """ 
    RECIBE: 1 o 0
    DEVUELVE: labelsList[0] o labelsList[1] respectivamente
    """  
    
    if binaryIndicator == 1:
        return labelsList[0]
    if binaryIndicator == 0:
        return labelsList[1]
    else:
        return None

In [164]:
def getAllConstantSum(k,n):

    """ 
    RECIBE: k es el valor a sumar de forma constante, n es la cantidad de numeros a sumar
    DEVUELVE: Todas las listas de n numeros que sumados dan k 
    """ 
    
    if n == 1:
        return [[k]]
    
    sums = []
    
    for i in range(k + 1):
        
        subsums = getAllConstantSum(k - i, n -1)
        
        for subsum in subsums:
            sums.append([i] + subsum)
    
    return sums

In [165]:
def isIn(element,container):

    """ 
    RECIBE: Un elemento y un contenedor de elementos
    DEVUELVE: Si el elemento esta o no en el contenedor
    """ 
    
    return element in container

In [166]:
def createLengthDic(words):
    
    """ 
    RECIBE: Un conjunto de palabras
    DEVUELVE: Un diccionario que tiene como key longitudes y values listas de palabras con esas longitudes
    EJEMPLO: words = ['a','b','aa','bbb'] ---> {1:['a','b'], 2:['aa'], 3:['bbb']}
    """ 
    
    d = {}
    
    for word in words:
        n = len(word)
        
        addToDic(d,n,[word])
    
    return d

In [167]:
def makeSortedLengthValues(queryWord,words,lengths):
    
    """ 
    RECIBE: Una palabra sobre la que se va a comparar longitudes, un conjunto de palabras
            y el conjunto de las longitudes que poseen esas palabras recibidas
    DEVUELVE: Una lista ordenada de menor a mayor que tiene como elementos las diferencias
                (no en modulo) de las longitudes entre cada palabra en words y la queryWord
    EJEMPLO: query = 'aa', words = ['a','aa','aaa','aaaa'], lengths = [1,2,3,4] -----> [0,-1,1,2]
    """ 
    
    temp_lengthCandidates = {length - len(queryWord) : abs(length - len(queryWord)) for length in lengths}
    lengthCandidates = [length - len(queryWord) for length in lengths]
    return sorted(lengthCandidates, key = temp_lengthCandidates.get)

In [168]:
def capitalizeText(text):
    
    """ 
    RECIBE: Un texto
    DEVUELVE: Ese texto pero con mayusculas al principio de cada palabra
    """
    
    tokens = text.split(' ')
    capitalized = []
    
    for token in tokens:
        capitalized.append(token.capitalize())
    return ' '.join(capitalized)

In [169]:
def reverseDic(dic):
    
    """ 
    RECIBE: Un diccionario de tipo k : [valores en iterable]
    DEVUELVE: Un dicciorio que tiene como claves los valores y como valores las claves 
            (es tipo indice invertido)
    """
    
    reversedDic = {}
    
    for k in dic:
        for v in dic[k]:
            addToDic(reversedDic, v, [k])
            
    for v in reversedDic:
        reversedDic[v] = ' '.join(list(set(reversedDic[v])))
        
    return reversedDic

In [170]:
def hasValueInColumn(tweet,value,column):
    
    """ 
    RECIBE: Un tweet un valor y una columna
    DEVUELVE: Si ese tweet tiene ese valor en esa columna
    """    
    
    return tweet[column] == value

In [171]:
def isDisaster(tweet):
    
    """ 
    RECIBE: Un tweet
    DEVUELVE: Si ese tweet es desastre o no
    """    
    
    return hasValueInColumn(tweet,1,'target')

# PERSISTENCIA (CARGA Y GUARDADO)

In [1]:
def generateAbsolutePath(fileName, exists = False, nombreRepo = 'DatosRepo'):
    
    """ 
    RECIBE: El nombre de un archivo
            exists es para indicar que el archivo ya existe. En cuyo caso lo va a buscar a donde este
    DEVUELVE: El path absoluto del archivo
    TENER EN CUENTA: Si se pone mal escrito el archivo es posible que lo encuentre igual por ej 'trai' en vez de 'train.csv'
                     Asume que tu repo se llama DatosRepo a menos que se lo digas en <nombreRepo>
    EJEMPLO: 'miArchivo.txt' -----> '/home/axelmpm/DatosRepo/miArchivo.txt'
    """
    
    path = '/home/' + getpass.getuser() + '/' + nombreRepo
    
    if exists:
        for dirpath, dirnames, filenames in os.walk(path):
            for filename in [f for f in filenames if (os.path.basename(f) == fileName)]:
                return os.path.join(dirpath, filename)
    else:
        return os.path.abspath("") + '/' + fileName

In [2]:
def pathExists(fileName):
 
    """ 
    RECIBE: El nombre de un archivo
    DEVUELVE: Si el path absoluto asociado existe o no
    """

    return path.exists(generateAbsolutePath(fileName))

In [173]:
def readDic(fileName, transformStringedListToList = False):
    
    """ 
    RECIBE: El nombre de un archivo .csv
    DEVUELVE: El diccionario que estaba almacenado en disco
    """
    
    df = pd.read_csv(generateAbsolutePath(fileName + '.csv'))
    
    if transformStringedListToList:
        df['values'] = df.apply(lambda row : ast.literal_eval(row['values']), axis = 1)
    
    dic = {}
    
    keys = list(df['keys'])
    values = list(df['values'])
    
    for i in range(len(keys)):
        dic[keys[i]] = values[i]
    
    return dic

In [174]:
def getWordInListFromFile(fileName):
    
    """ 
    RECIBE: El nombre de un archivo donde cada linea tiene palabras
    DEVUELVE: Una lista donde cada elemento son las palabras que estaban como linea en el archivo (no toma las palabras de longitud 1)
    """ 
    
    file = open(generateAbsolutePath(fileName),'r')
    
    l = []
    
    for line in file:
        if len(line[:-1]) > 1:
            l.append(line[:-1]) #le saco el \n
    file.close()
    return l

In [175]:
def readFromFileIntoList(fileName, strings = False):
    
    """ 
    RECIBE: Un nombre de archivo que se supone que almacena una lista previamente almacenada 
            con el metodo listToFile
    DEVUELVE: La lista guardada en ese archivo
    """
    
    file = open(generateAbsolutePath(fileName),'r')
    
    strList = (file.read()).split('&&&')[:-1]
    
    recoveredList = []
    
    for e in strList:
        
        if strings:
            recoveredList.append(e)
        else:
            recoveredList.append(ast.literal_eval(e))
    file.close()
    return recoveredList

In [176]:
def listToFile(aList,fileName):

    """ 
    RECIBE: Un nombre para el archivo que va a almacenar la lista y 
            una lista a almacenar
    """
    
    file = open(generateAbsolutePath(fileName),'w')
    
    for e in aList:
        file.write(str(e) + '&&&')
        
    file.close()    

# DICCIONARIOS Y DATAFRAMES

In [223]:
def getDataFrameOfFrecuencies(elementsLabel,elements, sort = True, sortAscending = False):
    
    """ 
    RECIBE: Una lista o secuencia de elementos y un nombre para estos
    DEVUELVE: Un dataframe con dos columnas, una de labels y otro de las frecuencias
    """ 
    
    temp_elements = list(elements.copy())
    
    df = pd.DataFrame({elementsLabel : temp_elements, 'Cantidad' : [1 for i in list(range(len(temp_elements)))]})
    df = df.groupby(elementsLabel).sum().reset_index()
    
    if sort:
        df.sort_values('Cantidad', ascending = sortAscending, inplace = True)
    return df

In [178]:
def dicToDataFrame(dic, keysLabel = "keys", valuesLabel = "values", sort = True, sortAscending = False):
    
    """ 
    RECIBE: Un diccionario y nombres para sus claves y valores y si si quiere ordenar o no ascendente o no
    DEVUELVE: Un dataframe con dos columnas, una de keys y otro de las values
    """
    
    df = pd.DataFrame({keysLabel : list(dic.keys()), valuesLabel : list(dic.values())})
    
    if sort:
        df.sort_values(valuesLabel, ascending = sortAscending, inplace = True)
    return df

In [179]:
def barplot(x,y,data, hue = None, xLabel = None,yLabel = None,title = '',
            palette = 'Spectral',figX = 25,figY = 10,fontSize = 1.7, xrotate = 0):
    
    '''
    RECIBE:
        x: es el label de la columna con los datos de x del dataFrame 'data'
        y: es el label de la columna con los datos de y del dataFrame 'data'
        data: es el dataframe de donde sacar los datos
        xLabel: es el label para x en el plot
        yLabel: es el label para y en el plot
        xLabel: es el titulo del plot
        palette: es la paleta de colores a usar
        figX: es el tamaño en x del plot
        figY: es el tamaño en y del plot
        fontSize: es el tamaño del font en el plot
    A TENER EN CUENTA: No afecta al dataframe 'data'
    DEVUELVE: El plot
    '''
    
    if xLabel == None:
        xLabel = x
    
    if yLabel == None:
        yLabel = y
    
    sns.set(rc = {'figure.figsize':(figX,figY)})
    sns.set(font_scale = fontSize)
    v = sns.barplot(x = x, y = y, data = data, 
                 palette = palette, saturation = 1, hue = hue)
    
    v.set(xlabel = xLabel, ylabel= yLabel)
    v.set(title = title)
    for item in v.get_xticklabels():
        item.set_rotation(xrotate)
    
    return v

# FUNCIONES RELACIONADAS A LA PROBABILIDAD

In [15]:
def probabilitiesOfGivenX(eventA,randomVariableX,data):
    
    """ 
    RECIBE: Un data frame, un evento A y una variable aleatoria que es una columna en el data frame
    DEVUELVE: Devuelve la probabilidad condicional P(A| X = x) tomando como espacio muestral equiprobable el
              dataframe
    """ 
    
    probabilityMap = {}
    for index, row in data.iterrows():
        
        if not row[randomVariableX] in probabilityMap:
            probabilityMap[row[randomVariableX]] = probabilityOfGivenThat(eventA,
                                                                         lambda x : x[randomVariableX] == row[randomVariableX],
                                                                          data)
            
    return probabilityMap

In [14]:
def probabilitiesOfGivenXY(eventA,randomVariableX,randomVariableY,data):
    
    """ 
    RECIBE: Un data frame, un evento A y dos variables aleatorias que son columnas en el data frame
    DEVUELVE: Devuelve la probabilidad condicional P(A| X = x, Y = y) tomando como espacio muestral equiprobable el
              dataframe
    """ 

    probabilityMap = {}
    for index, row in data.iterrows():
        
        if not row[randomVariableX] in probabilityMap:
            if not row[randomVariableY] in probabilityMap[row[randomVariableX]]:
                probabilityMap[row[randomVariableX]] = {[row[randomVariableY]] : probabilityOfGivenThat(eventA,
                                                                             lambda x : x[randomVariableX] == row[randomVariableX] and x[randomVariableY] == row[randomVariableY],
                                                                              data)}
            
    return probabilityMap
    
    

In [181]:
def probabilityOfGivenThat(eventA,eventB,data):
    
    """ 
    RECIBE: Un data frame y dos funciones booleanas que dado una row del data frame deciden corresponden
            o no a los eventos o condiciones A y B
    DEVUELVE: Devuelve la probabilidad condicional P(A|B) tomando como espacio muestral equiprobable el
              dataframe
    """           
    
    frequencyOfAandB = data[ data.apply(lambda row : eventA(row) and eventB(row), axis = 1)]['id'].count()
    frequencyOfB = data[ data.apply(lambda row : eventB(row), axis = 1)]['id'].count()
    
    return frequencyOfAandB/frequencyOfB

In [182]:
def expectedValue(randomVariable,data):
    
    """ 
    RECIBE: El data set y una randomVariable que es una columna de dataframe que a cada tweet le asigna un valor de algun tipo
    DEVUELVE: Devuelve el la esperanza de la variable aleatoria pasada por parametro
    """
    
    data_c = data.copy()
    data_c['frec'] = 1
    frecs = data.groupby(randomVariable).sum().reset_index()[[randomVariable,'frec']]
    
    value = 0
    for index, row in frecs.iterrows():
        value += (row[randomVariable] * row['frec'])
        
    return value/data_c.shape[0]
    

# FUNCIONES DE SIMILARIDAD EN TEXTOS

In [183]:
def getHammingSimilarity(word1,word2):
    
    """ 
    RECIBE: Dos palabras de igual longitud
    DEVUELVE: Un numero del 0 al 1 donde 1 significa que son el mismo string y 0 es que no se parecen en nada
    EJEMPLO: word1 = casa, word2 = capa ---->  0.75 (porque tienen en comun la primera c, la segunda a
             y la ultima a, osea 3/4 bien)
    """     
    
    length = len(word1)
    mismatches = 0
    
    for i in range(length):
        if word1[i] != word2[i]:
            mismatches += 1
    return 1 - mismatches/length
    

In [184]:
def generateWordExtentionsIntoNChars(word,n,extentionChar):
    
    """ 
    RECIBE: Una palabra a extender, la cantidad de caracteres que debe tener la version extendida y que caracter usar al extender
    DEVUELVE: Devuelve un set de todas las posibles extensiones
    EJEMPLO: word = 'casa', n = 6, extentionChar = '.' ---->
                {'..casa','.c.asa','.ca.sa','.cas.a','.casa.','c..asa','c.a.sa','c.as.a','c.asa.','ca..sa'
                    'ca.s.a','ca.sa.','cas..a','cas.a.','casa..'}   
    """
    
    if len(word) == '':
        return [extentionChar * n]
    
    if (len(word) > n):
        return [word]
    
    if len(word) == n:
        return [word]
    
    wordLength = len(word)
    amountOfCharsToAdd = n - wordLength
    
    extentions = set()
    
    amounts = getAllConstantSum(amountOfCharsToAdd,wordLength + 1)
    
    for amountOfExtendingChars in amounts:

        extention = ''
        for i in range(len(word)):
            extention += (extentionChar * amountOfExtendingChars[i]) + word[i]
        extention += extentionChar * amountOfExtendingChars[-1]
        
        extentions.add(''.join(list(extention)))
    
    return extentions

In [185]:
def getSimilarity(text1,text2):
    
    """ 
    RECIBE: Dos textos de cualquier longitud (no muy largos)
    DEVUELVE: Un numero del 0 al 1 donde 1 significa que son el mismo string y 0 es que no se parecen en nada
    A TENER EN CUENTA: Usa la similaridad de Hamming
                       Es mas o menos costosa a partir de 20 caracteres de texto aprox
                       Remueve los espacios por lo que no hace falta preprocesar texto
    EJEMPLO: word1 = casa, word2 = ca ---->  0.5 (porque tienen en comun la primera c, la segunda a, osea 2/4 bien)
    """ 
    
    EMPTY_CHARACTER = ' '
    
    if text1 + text2 == '':
        return 1
    
    if text1 + text2 == text1 or text1 + text2 == text2:
        return 0
    
    splitedText1 = text1.split(' ')
    splitedText2 = text2.split(' ')
    
    concatenatedWord1 = (''.join(splitedText1)).lower()
    concatenatedWord2 = (''.join(splitedText2)).lower()
    
    if len(concatenatedWord1) > len(concatenatedWord2):
        longest = concatenatedWord1
        shortest = concatenatedWord2
    else:
        longest = concatenatedWord2
        shortest = concatenatedWord1
        
    lenDif = len(longest) - len(shortest)
    maxSimilarity = 0
    extentions = generateWordExtentionsIntoNChars(shortest,len(longest),EMPTY_CHARACTER)
    
    for extention in extentions:
        
        subSimilarity = getHammingSimilarity(extention,longest)
        
        if (subSimilarity >= maxSimilarity):
            maxSimilarity = subSimilarity
            
    return maxSimilarity
        

In [186]:
def getMaxSimilarityWord(queryWord, words, threshold = 1, lengthDic = None):

    """ 
    RECIBE: El una palabra y palabras a comparar
    TENER EN CUENTA: Si la palabras es vacia retorna la palabra vacia
                    La lista de words debe ser chica porque esto es muy costoso
                    busca hasta encontrar una palabra que supera el threshold
    DEVUELVE: La palabra que mas se parece y su similitud
    """
    
    if len(words) == 0:
        return queryWord

    if queryWord == '':
        return queryWord
    
    maxSimilarity = 0
    mostSimilarKnownWord = words[0]
    
    AMOUNT_COSIDERED_LITTLE = 100
    
    if len(words) < AMOUNT_COSIDERED_LITTLE:
        THRESHOLD_SEARCH = len(words)
    else:
        THRESHOLD_SEARCH = AMOUNT_COSIDERED_LITTLE + len(words)/10
    
    if queryWord in words:
        return queryWord
    
    if lengthDic == None:
        lengthDic = createLengthDic(words)
        
    candidateLengthDiffs = makeSortedLengthValues(queryWord, words, lengthDic)
    
    found = False
    searchLimitReached = False
    searched = 0
    
    for lengthDiff in candidateLengthDiffs:
        
        if (len(queryWord) + lengthDiff) in lengthDic:
            candidates = lengthDic[len(queryWord) + lengthDiff]
        else:
            continue
        
        for knownWord in candidates:

            similarity = getSimilarity(queryWord,knownWord)

            if maxSimilarity <= similarity:

                maxSimilarity = similarity
                mostSimilarKnownWord = knownWord

            if similarity >= threshold:
                found = True
                break
                
            searched += 1
            
            if searched >= THRESHOLD_SEARCH:
                searchLimitReached = True
                break
                
        if found or searchLimitReached:
            break
    
    return mostSimilarKnownWord, maxSimilarity

In [187]:
def isSimilar(text1, text2, threshold = 0.7):

    """ 
    RECIBE: Dos textos (no muy grandes)
    DEVUELVE: Si la similitud es mayor a un threshold
    """ 
    
    return getSimilarity(text1,text2) >= threshold

# FUNCIONES DE LOCATION

In [188]:
def isAKnownPlace(token):

    """
    RECIBE: Un token cualquiera
    A TENER EN CUENTA: Supone que esta bien escrito
    DEVUELVE: Si el token es una ciudad o pais o continente y a que pertenece o UNKNOWN
    """
    
    capitalizedVersion = capitalizeText(token)
    
    if token in citiesAndUsStatesNames:
        return True, CITY, token
    else:
        if capitalizedVersion in citiesAndUsStatesNames:
            return True, CITY, capitalizedVersion
    
    if token in countriesNames:
        return True, COUNTRY, token
    else:
        if capitalizedVersion in countriesNames:
            return True, COUNTRY, capitalizedVersion
    
    if token in continentsNames:
        return True, CONTINENT, token
    
    else:
        if capitalizedVersion in continentsNames:
            return True, CONTINENT, capitalizedVersion
        
    return False, UNKNOWN, UNKNOWN

In [189]:
def getCityOrStateByAcronimsAndCountry(acronims, country):
    
    """ 
    RECIBE: Una lista de acronimos a interpretar como ciudades y un pais que debe contener a esa ciudad
    DEVUELVE: El nombre de la ciudad completo que esta contenido en el pais pasado o en caso contrario
              UNKNOWN
    """ 
    
    for acr in acronims:
        if acr in cityOrUSStateAcronimToNameMap:
            asociatedCities = cityOrUSStateAcronimToNameMap[acr]
            
            for city in asociatedCities:
                asociatedCountries = citiyOrUSStateNameToCountryMap[city]

                for c in asociatedCountries:

                    if country == c or country in c:
                        return city
                
    return UNKNOWN    

In [190]:
def getCountryByAcronimsAndContinent(acronims, continent):
    
    """ 
    RECIBE: Una lista de acronimos a interpretar como paises y un continent que debe contener a esa pais
    DEVUELVE: El nombre de el pais completo que esta contenido en el continente pasado o en caso contrario
              UNKNOWN
    """ 
    
    for acr in acronims:
        if acr in countryAcronimToCountryName:
            asociatedCountries = countryAcronimToCountryName[acr]

            for country in asociatedCountries:

                asociatedContinent = countryNameContinentNameMap[country]
                if continent == c or continent in c:
                    return country
                
    return UNKNOWN    

In [191]:
def getCountry(token, byCity = False):
    
    """ 
    RECIBE: Un token y byCity que indica si el token es una ciudad de ese pais o no
    DEVUELVE: La version del token en forma de pais (si existe) y si no devuelve UNKNOWN
              En caso de byCity = True retorna el pais que contiene esa city 
              (ese token como city del pais)
    """
    
    if byCity:
        return cityOrStateToCountryMap[token][0]
    
    else:
        return token

In [192]:
def getContinent(token, byCountry = False):
    
    """ 
    RECIBE: Un token y byCity que indica si el token es un pais de ese continente o no
    DEVUELVE: La version del token en forma de continente (si existe) y si no devuelve UNKNOWN
              En caso de byCity = True retorna el continente que contiene ese pais
            (ese token como pais del continente)
    """
    
    if byCountry:
        return countryNameContinentNameMap[token]
    
    else:
        return token

In [193]:
def tokenizeLocation(text):
    
    """ 
    RECIBE: Texto crudo de una location
    DEVUELVE: Una lista de 'palabras' (tokens) que por su 'significado' se supone que son entidades
              separadas
    """
    
    return separateOutBiaSpecialCaractersAndMayuscFromText(str(text), notRemove = ['.'])

In [194]:
def getLocation(rawLocation, threshold = 0.9):

    """ 
    RECIBE: Texto crudo de una location
    TENER EN CUENTA: Tomamos como city a cualquier 'cosa' (ciudad, provincia, etc) 
                     que este dentro de un pais
                     Asume que la gente escribe bien los lugares
    DEVUELVE: Una tupla del tipo (continent,country,city)
    """
    
    if rawLocation == None or str(rawLocation) == 'nan' :
        return (UNKNOWN,UNKNOWN,UNKNOWN)
    
    tokens = tokenizeLocation(rawLocation)
    
    city = UNKNOWN
    country = UNKNOWN
    continent = UNKNOWN
    
    cityFound = False
    countryFound = False
    continentFound = False
    
    if MAX_LOCATION_LENGTH_IN_WORDS > len(tokens):
        maxWindowLength = len(tokens)
    else:
        maxWindowLength = MAX_LOCATION_LENGTH_IN_WORDS

    acronims = []
    
    for window in range(maxWindowLength, 0, -1):
        
        identified = []
        i = 0
        while (i < (len(tokens) - window + 1) and (not cityFound or not countryFound or not continentFound)):
            #print()
            #print("WINDOW: {}".format(window))
            #print(i)
            #print("COND1: {} | COND2: {}".format(i < (len(tokens) - window + 1),(not cityFound or not countryFound or not continentFound)))
            #print("TOKENS = {}".format(tokens))
            subTokens = tokens[i : i + window]
            token = ' '.join(subTokens) 
            #print("TOKEN: {}".format(token))
            
            isKnown, placeType, knownVersion = isAKnownPlace(token) #no mira acronimos
            if isKnown:
                identified += subTokens
                
                if placeType == CITY:
                    cityFound = True
                    city = knownVersion
                    
                elif placeType == COUNTRY:
                    country = getCountry(knownVersion)
                    continent = getContinent(country, byCountry = True)
                    countryFound = True
                    continentFound = True
                    
                elif placeType == CONTINENT:
                    continent = getContinent(knownVersion)
                    continentFound = True
                    
            if isAcronim(token):
                acronims.append(token)
                        
            i += 1
            
        tokens = [t for t in tokens if not t in identified]
            
        if cityFound and countryFound and continentFound:
            break;
            
    if cityFound and not countryFound:
        country = getCountry(city, byCity = True)
        continent = getContinent(country, byCountry = True)
        countryFound = True
        continentFound = True
        
    if not cityFound and countryFound and continentFound:
        if len(acronims) > 0:
            city = getCityOrStateByAcronimsAndCountry(acronims,country)
            if city != UNKNOWN:
                cityFound = True
            
    if not countryFound and continentFound:
        if len(acronims) > 0:
            country = getCountryByAcronimsAndContinent(acronims,continent)
            countryFound = True
            
    if not cityFound and not countryFound and not continentFound:
        if len(acronims) > 0:
            for acr in acronims:
                if acr in cityOrUSStateAcronimToNameMap:
                    city = cityOrUSStateAcronimToNameMap[acr][0]
                    cityFound = True
                    break;            
        
        #print("TOKENS = {}".format(tokens))
    #print((continent,country, city))
    return (continent,country, city)
            

In [195]:
def haveAtLeastOneUnknown(parsedLocation):
    
    """ 
    RECIBE: Una tupla de location
    DEVUELVE: Si en esa tupla hay al menos un UNKNOWN
    """
    
    for component in parsedLocation:
        if component == UNKNOWN:
            return True
    return False

In [196]:
def haveAllUnknown(parsedLocation):
    
    """ 
    RECIBE: Una tupla de location
    DEVUELVE: Si en esa tupla todos son UNKNOWN
    """
    
    for component in parsedLocation:
        if component != UNKNOWN:
            return False
    return True

# FUNCIONES DE CONTEXTO

In [ ]:
def extractWordsFrequencies(targets,textsTokens,cleanWords,tokenCleanMap):
    
    """ 
    RECIBE: Una lista de tokens, unas lista de palabras traducidas de esos tokens (cleanWords)
            y un diccionario que mapea token--->clean y una lista de targets que se corresponde
            los textsTokens
    DEVUELVE: Un diccionario de frecuencia de palabras en el dataframe y otro de frecuencia en desastres
    """
    
    disasterWordFrequency = {}
    totalWordFrequency = {}
    
    i = 0
    for tokens in textsTokens:
        
        for token in tokens:
            
            word = tokenCleanMap[token]
            
            addToDic(totalWordFrequency,word,tokens.count(token))
            
            if targets[i] == 1:
                
                addToDic(disasterWordFrequency,word,tokens.count(token))
                
        i += 1
                    
    return disasterWordFrequency,totalWordFrequency

In [7]:
def addWordPairToContextualMap(contextualMap,word1,word2):
    
    if word1 in contextualMap:
        addToDic(contextualMap[word1],word2,1)
    else:
        addToDic(contextualMap,word1,{word2 : 1})   
    

In [197]:
def addContextualKnowledge(tokens,contextualMap,tokenToCleanMap):

    """ 
    RECIBE: Una lista de tokens, un contexto de textos previos y una 
            'traduccion' del token a su version limpia (la escencia)
    DEVUELVE: El contexto actualizado con el contenido del texto pasado
    """
    
    pairs = []
    
    for i in range(len(tokens)):
        for j in range(i,len(tokens)):
            if j < len(tokens):
                if tokens[i] in tokenToCleanMap and tokens[j] in tokenToCleanMap:
                    for w1 in tokenToCleanMap[tokens[i]]:
                        for w2 in tokenToCleanMap[tokens[j]]:
                            pairs.append((w1,w2))
    
    for pair in pairs:
            
        addWordPairToContextualMap(contextualMap,pair[0],pair[1])
        addWordPairToContextualMap(contextualMap,pair[1],pair[0])
            
    return contextualMap

In [198]:
def getContextualKnowledge(textsTokens,tokenToCleanMap):

    """ 
    RECIBE: Una lista de listas de tokens y una 'traduccion' del token a su version limpia (la escencia)
    DEVUELVE: Un diccionario llamado contexto que tiene como claves combinaciones de palabras que se dieron
                juntas en algun texto de la lista y cuya clave es la cantidad de textos en los que se dio
    """
    
    contextualMap = {}
    
    for tokens in textsTokens:
        
        contextualMap = addContextualKnowledge(tokens,contextualMap,tokenToCleanMap)
    
    return contextualMap

In [8]:
def getReinforcement(contextualMap,word1,word2):
    
    if word1 in contextualMap:
        if word2 in contextualMap[word1]:
            return contextualMap[word1][word2]
    return 0

# FUNCIONES DE NEGATIVIDAD

In [200]:
def getWordIntrinsicNegativity(word,disasterWordFrequency,totalWordFrequency):
    
    """ 
    RECIBE: Una palabra y los mapas de frecuencia en desastre y total
    DEVUELVE: La frecuencia de uso de word en desastres / la frecuencia de word en todo el data frame
    """
    
    if word in disasterWordFrequency and word in totalWordFrequency:
        return disasterWordFrequency[word] / totalWordFrequency[word]
    else:
        return 0

In [201]:
def calculateNegativityMap(words,disasterWordFrequency,totalWordFrequency):
   
    """ 
    RECIBE: Una lista de palabras procesadas y los mapas de frecuencia en desastre y total
    DEVUELVE: Un diccionario que tiene como key nouns y value su negatividad
    """
    negativityMap = {}
    
    for word in words:
        
        negativityMap[word] = getWordIntrinsicNegativity(word,disasterWordFrequency,totalWordFrequency)
        
    return negativityMap

In [202]:
def getContextualNegativity(queryWord,words,context):
    
    """ 
    RECIBE: Una palabra, una lista de palabras y un contexto
    DEVUELVE: La negatividad contextual por la relacion entre estas palabras
    """
    
    contextualNegativity = 0
    totalReinforcementMass = 0
    
    for word in words:
        
        reinforcement = getReinforcement(context.contextualMap,queryWord,word)
        
        asociates = [asociate for asociate in context.contextualMap[queryWord]]
        
        for asociate in asociates:
            
            totalReinforcementMass += getReinforcement(context.contextualMap,queryWord,asociate)
        
        contextualNegativity += reinforcement/totalReinforcementMass
            
    return contextualNegativity

In [9]:
def getTextNegativity(text,context):
    
    """ 
    RECIBE: Un texto y un context
    DEVUELVE: Un numero de que es mas grande cuando mas negativo es el tweet
    """
    
    tokens = text.split(' ')
    
    cleanedUpWords = []
    
    for token in tokens:
        
        if token in context.tokenCleanMap:
            cleanedUpWords += context.tokenCleanMap[token]
    
    intrinsicNegativity = 0
    contextualNegativity = 0
    totalIntrinsicNegativity = 0
    totalContextualNegativity = 0
    contextualPonderation = 0
    
    for word in cleanedUpWords: 
        
        if word in context.negativityMap:
            intrinsicNegativity = context.negativityMap[word]
            contextualNegativity =  getContextualNegativity(word,cleanedUpWords,context)

            if contextualNegativity > 0:
                contextualPonderation += 1

            totalIntrinsicNegativity += intrinsicNegativity
            totalContextualNegativity += contextualNegativity * intrinsicNegativity
    
    #print("TOTAL NEGATIVITY: {}, NEGATIVITY EXTRACTED FROM CONTEXT: {}".format(totalIntrinsicNegativity + totalContextualNegativity * contextualPonderation,totalContextualNegativity * contextualPonderation))
    return totalIntrinsicNegativity + totalContextualNegativity * contextualPonderation

In [204]:
def getTweetNegativity(tweet):
    
    """ 
    RECIBE: Un tweet
    DEVUELVE: Un numero de que es mas grande cuando mas negativo es el tweet
    """
    
    return getTextNegativity(extractTextFormTweet(tweet))

# FUNCIONES DE IMPORTANCIA DE TEXTO

In [205]:
def getTextImportance(text):
    
    """ 
    RECIBE: Un texto crudo
    DEVUELVE: Un numero de que es mas grande cuando mas importante es el tweet
    """
    
    importanceSigns = ['?','!']
    importanceCount = 0
    
    wordAmount = len(tokenize(text))
    
    for c in text:
        if c in importanceSigns or c.isupper():
            importanceCount += 1
            
    return importanceCount/wordAmount
    

In [206]:
def calculateTweetImportance(tweet):
    
    """ 
    RECIBE: Un tweet
    DEVUELVE: Un numero de que es mas grande cuando mas importante es el tweet
    """
    
    return getTextImportance(extractTextFormTweet(tweet))

# CLASE CONTEXT

In [10]:
class Context:
    
    """ 
    Clase que engloba toda las operaciones y nociones de contexto de un texto
    """
    
    def __init__(self,data = None, similarityThreshold = 0.8, load = ''):
        
        """ 
        RECIBE: 
            texts es una lista de textos (tweets) a entender
            similarityThreshold es una tolerancia para la similaridad. 
                                Si dos palabras superan ese limite se consideran simialres
            load es opcional y es el nombre del archivo que almacena
                                el contextualMap guardado en disco en caso de haberlo hecho
        """
        
        self.stopWords = set(stopwords.words('english'))
        self.similarityThreshold = similarityThreshold
        
        self.__atrbt_texts = load + '_' + 'texts'
        self.__atrbt_textsTokens = load + '_' + 'textsTokens'
        self.__atrbt_cleanWords = load + '_' + 'cleanWords'
        
        self.__atrbt_cleanTokenMap = load + '_' + 'cleanTokenMap'
        self.__atrbt_tokenCleanMap = load + '_' + 'tokenCleanMap'
        
        self.__atrbt_contextualMap = load + '_' + 'contextualMap'
        self.__atrbt_disasterCleanWordFrequency = load + '_' + 'disasterCleanWordFrequency'
        self.__atrbt_totalCleanWordFrequency = load + '_' + 'totalCleanWordFrequency'
        self.__atrbt_negativityMap = load + '_' + 'negativityMap'
        
        
        #====================================================
        self.data = data
        print("Data loaded")
        
        #====================================================
        if pathExists(self.__atrbt_texts):
            self.texts = readFromFileIntoList(self.__atrbt_texts, strings = True)
            print("Texts loaded")
        else:
            self.texts = extractTextFrom(data)
            print("Extracted texts")
        
        #====================================================
        if pathExists(self.__atrbt_textsTokens):
            self.textsTokens = readFromFileIntoList(self.__atrbt_textsTokens, strings = True)
            print("Texts Tokens loaded")
        else:
            self.textsTokens = [tokenize(text) for text in self.texts]
            print("Extracted Texts Tokens")
        
        #====================================================
        if pathExists(self.__atrbt_cleanWords) and pathExists(self.__atrbt_tokenCleanMap + '.csv') and pathExists(self.__atrbt_cleanTokenMap + '.csv'):
            
            self.cleanWords = readFromFileIntoList(self.__atrbt_cleanWords, strings = True)
            print("Clean Words loaded")
            
            self.tokenCleanMap = readDic(self.__atrbt_tokenCleanMap, transformStringedListToList = True)
            print("Token Clean Words map loaded")
            
            self.cleanTokenMap = readDic(self.__atrbt_cleanTokenMap, transformStringedListToList = True)
            print("Clean Words Token map loaded")
            
        else:
            self.cleanWords, self.tokenCleanMap, self.cleanTokenMap = cleanUpTokens(self.textsTokens)
            print("Tokens processed into cleaned up words and mapped")
            
        #====================================================
        if pathExists(self.__atrbt_contextualMap + '.csv'):
            self.contextualMap = readDic(self.__atrbt_contextualMap, transformStringedListToList = True)
            print("Contextual map loaded")
        else:
            self.contextualMap = getContextualKnowledge(self.cleanWords,self.tokenCleanMap)
            print("Contextual map produced")
        
        #====================================================
        if pathExists(self.__atrbt_disasterCleanWordFrequency + '.csv') and pathExists(self.__atrbt_totalCleanWordFrequency + '.csv'):
            
            self.disasterCleanWordFrequency = readDic(self.__atrbt_disasterCleanWordFrequency)
            print("Disaster Frequency map loaded")
            
            self.totalCleanWordFrequency = readDic(self.__atrbt_totalCleanWordFrequency)
            print("Total Frequency map loaded")
        else:
            self.disasterCleanWordFrequency, self.totalCleanWordFrequency = extractWordsFrequencies(list(self.data['target']),self.textsTokens,self.cleanWords,self.tokenCleanMap)
            print("Total and Disaster frequencies calculated for every word")
        
        #====================================================
        if pathExists(self.__atrbt_negativityMap + '.csv'):
            self.negativityMap = readDic(self.__atrbt_negativityMap)
            print("Negativity map loaded")
        else:
            self.negativityMap = calculateNegativityMap(self.cleanWords,self.disasterCleanWordFrequency,self.totalCleanWordFrequency)
            print("Negativity map produced")
        
        
    def related(self, word, r = 1, restrict = True):
        
        """ 
        RECIBE: La palabra de la que se quiere conocer sus asociados y el orden minimo de su asociacion (su reinforcement)
                restric es para quedarse solo con las relaciones con palabras filtradas
        DEVUELVE: Una lista de todos los asociados (palabras, orden) que cumplen con el r
        """
        
        if word in cleanWords:
            asociates = [asociate for asociate in context.contextualMap[word]]
        
        elif word in tokenCleanMap:
            asociates = [asociate for asociate in context.contextualMap[tokenCleanMap[word]]]
        
        else:
            asociates = [asociate for asociate in tokenCleanMap[getMaxSimilarityWord(word,list(self.tokenCleanMap.keys()))[0]]]

        if r == 1:
            return asociated
        
        stronglyAsociated = []
        
        for pair in asociated:
            if pair[1] >= r:
                stronglyAsociated.append(pair)
                                
        return stronglyAsociated
    
    
    def save(self,name):
        
        """ 
        RECIBE: El nombre de archivo que guarda el contextualMap
        DEVUELVE: Guarda el contextualMap en un csv
        """
        
        if name == '':
            return None
    
        
        #====================================================
        if not pathExists(self.__atrbt_texts):
            
            listToFile(self.texts, name + '_' + 'texts')
            print("Texts saved")
            
        #====================================================
        if not pathExists(self.__atrbt_textsTokens):
            
            listToFile(self.textsTokens, name + '_' + 'tokens') 
            print("Texts Tokens saved")
        
        #====================================================
        if not pathExists(self.__atrbt_cleanWords):
            
            listToFile(self.cleanWords, name + '_' + 'cleanWords') 
            print("Clean Words saved")
        
        #====================================================
        if not pathExistss(self.__atrbt_tokenCleanMap + '.csv'):
            
            listToFile(self.tokenCleanMap).to_csv(name + '_' + 'tokenCleanMap' + '.csv') 
            print("Token Clean Words map saved")
        
        #====================================================
        if not pathExists(self.__atrbt_cleanTokenMap + '.csv'):
            
            dicToDataFrame(self.cleanTokenMap).to_csv(name + '_' + 'cleanTokenMap' + '.csv') 
            print("Clean Words Token map saved")
            
        #====================================================
        if not pathExists(self.__atrbt_disasterCleanWordFrequency + '.csv'):
            
            dicToDataFrame(self.disasterCleanWordFrequency).to_csv(name + '_' + 'disasterCleanWordFrequency' + '.csv')
            print("Disaster Frequency map saved")
        
        #====================================================
        if not pathExists(self.__atrbt_totalCleanWordFrequency + '.csv'):
            
            dicToDataFrame(self.totalCleanWordFrequency).to_csv(name + '_' + 'totalCleanWordFrequency' + '.csv')
            print("Total Frequency map saved")
        
        #====================================================
        if not pathExists(self.__atrbt_negativityMap + '.csv'):
            
            dicToDataFrame(self.negativityMap).to_csv(name + '_' + 'negativityMap' + '.csv')
            print("Negativity map saved")
        
        #====================================================
        if not pathExists(self.__atrbt_contextualMap + '.csv'):
            
            dicToDataFrame(self.contextualMap).to_csv(name + '_' + 'contextualMap' + '.csv')
            print("Contextual map saved")
            